# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

from cassandra.cluster import Cluster

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [2]:
# Consolidate all CSV files into one     
def consolidate_csvs():
    # Get current folder and subfolder event data
    filepath = os.getcwd() + '/event_data'

    # collect and join filepaths
    for root, dirs, files in os.walk(filepath):
        file_path_list = glob.glob(os.path.join(root,'*'))
    
    full_data_rows_list = [] 
    for f in file_path_list:
        with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
            csvreader = csv.reader(csvfile) 
            next(csvreader)
            for line in csvreader:
                full_data_rows_list.append(line) 

    csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

    with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
        writer = csv.writer(f, dialect='myDialect')
        writer.writerow(['artist','first_name','gender','item_in_session','last_name','length',\
                    'level','location','session_id','song','user_id'])
        for row in full_data_rows_list:
            if (row[0] == ''):
                continue
            writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

# Part II. Complete the Apache Cassandra coding portion of your project. 

Now you are ready to work with the CSV file titled `event_datafile_new.csv`, located within the Workspace directory.  The `event_datafile_new.csv` contains the following columns: 

- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the `event_datafile_new.csv` after the code above is run:

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

In [3]:
# Column name to number map
CSV_COLS = {"artist": 0, "first_name": 1, "gender":2, "item_in_session": 3,"last_name": 4,
            "length": 5,"level": 6,"location": 7,"session_id": 8,"song": 9,"user_id": 10}

In [4]:
"""HELPER METHODS"""

def connect_to_cluster(host='127.0.0.1'):
    try:
        cluster = Cluster([host])
        return (cluster, cluster.connect())
    except Exception as e:
        print(e)

def create_keyspace(session, keyspace, repl_class='SimpleStrategy', repl_factor=1):
    replication = {'class' : repl_class, 'replication_factor' : repl_factor}
    try:
        session.execute(f"CREATE KEYSPACE IF NOT EXISTS sparkify WITH REPLICATION = {replication}")
    except Exception as e:
        print(e)
        
def set_keyspace(session, keyspace):
    try:
        session.set_keyspace(keyspace)
    except Exception as e:
        print(e)
        
def execute_query(session, sql):
    try:
        return session.execute(sql)
    except Exception as e:
        print(e)
        
def pandas_factory(colnames, rows):
    return pd.DataFrame(rows, columns=colnames)

def build_df(session, query):
    session.row_factory = pandas_factory
    result = session.execute(query, timeout=None)
    return result._current_rows

def build_insert_statment(table, columns):
    """Creates an insert statement for a given table and columns"""
    return f"""
    INSERT INTO {table} ({(', '.join(columns))})
    VALUES ({(', '.join(['%s'] * len(columns)))})
    """

In [5]:
cluster, session = connect_to_cluster()

def main(): 
    consolidate_csvs()
    create_keyspace(session, 'sparkify')
    set_keyspace(session, "sparkify")

if __name__ == '__main__':
    main()

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [6]:
drop_tbl = "DROP TABLE IF EXISTS music_history;"
create_tbl = """
CREATE TABLE music_history (
  artist text,
  song text,
  length float,
  session_id int,
  item_in_session int,
  PRIMARY KEY ((session_id, item_in_session), artist, song)
);
"""
execute_query(session, drop_tbl)
execute_query(session, create_tbl)

In [7]:
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = build_insert_statment('music_history', ['artist', 'song', 'length', 'session_id', 'item_in_session'])
        session.execute(query, (line[CSV_COLS['artist']], line[CSV_COLS['song']], float(line[CSV_COLS['length']]), int(line[CSV_COLS['session_id']]), int(line[CSV_COLS['item_in_session']])))
        

#### Do a SELECT to verify that the data have been inserted into each table

In [8]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
query = """
SELECT artist, song, length
FROM music_history  
WHERE session_id = 338  
AND item_in_session = 4;
"""

In [9]:
df = build_df(session, query)
df.head()

,artist,song,length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [10]:
## TO-DO: Query 2: Give me only the following: name of artist, song 
## (sorted by itemInSession) and user (first and last name)
## for userid = 10, sessionid = 182
drop_tbl = "DROP TABLE IF EXISTS music_history2;"
create_tbl = """
CREATE TABLE music_history2 (
  artist text,
  song text,
  session_id int,
  item_in_session int,
  user_id int,
  first_name text,
  last_name text,
  PRIMARY KEY (user_id, session_id, item_in_session)
);
"""
execute_query(session, drop_tbl)
execute_query(session, create_tbl)

In [11]:
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = build_insert_statment('music_history2', ['artist', 'song', 'session_id', 'item_in_session', 'user_id', 'first_name', 'last_name'])
        session.execute(query, (line[CSV_COLS['artist']], 
                                line[CSV_COLS['song']], 
                                int(line[CSV_COLS['session_id']]), 
                                int(line[CSV_COLS['item_in_session']]), 
                                int(line[CSV_COLS['user_id']]), 
                                line[CSV_COLS['first_name']], 
                                line[CSV_COLS['last_name']]))

In [12]:
query = """
SELECT artist, song, first_name, last_name
FROM music_history2
WHERE user_id = 10 
AND session_id = 182
"""

In [13]:
df = build_df(session, query)
df.head()

,artist,song,first_name,last_name
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


In [14]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history 
## who listened to the song 'All Hands Against His Own'
drop_tbl = "DROP TABLE IF EXISTS music_history3;"
create_tbl = """
CREATE TABLE music_history3 (
  song text,
  first_name text,
  last_name text,
  PRIMARY KEY (song, first_name, last_name)
);
"""
execute_query(session, drop_tbl)
execute_query(session, create_tbl)

In [15]:
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = build_insert_statment('music_history3', ['song', 'first_name', 'last_name'])
        session.execute(query, (line[CSV_COLS['song']], 
                                line[CSV_COLS['first_name']], 
                                line[CSV_COLS['last_name']]))

In [16]:
query = """
SELECT first_name, last_name 
FROM music_history3
WHERE song = 'All Hands Against His Own'
"""

In [17]:
df = build_df(session, query)
df.head()

,first_name,last_name
0,Jacqueline,Lynch
1,Sara,Johnson
2,Tegan,Levine


### Drop the tables then close out the sessions

In [18]:
## TO-DO: Drop the table before closing out the sessions

In [19]:
execute_query(session, "DROP TABLE music_history;")
execute_query(session, "DROP TABLE music_history2;")
execute_query(session, "DROP TABLE music_history3;")

In [20]:
session.shutdown()
cluster.shutdown()